In [ ]:
def fourier(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    dft = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    out = 20*np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))
    return out

film_landmarks = extract_landmarks(load_tsv(film_path), '29@[2479]|30@[34]', 6)
film_image = cv2.imread(film_img_path)
photo_image = cv2.imread(image_path)

out1 = fourier(film_image)
out2 = fourier(photo_image)

plt.figure()
plt.subplot(211)
plt.imshow(out1, cmap='gray')
plt.subplot(212)
plt.imshow(out2, cmap='gray')
plt.show()


orb1 = cv2.ORB_create()
# 키 포인트 검출과 서술자 계산
keypoints1, descriptor1 = orb1.detectAndCompute(out1, None)


orb2 = cv2.ORB_create()
# 키 포인트 검출과 서술자 계산
keypoints2, descriptor2 = orb1.detectAndCompute(out2, None)


# 인덱스 파라미터 설정 ---①
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH,
                   table_number = 6,
                   key_size = 12,
                   multi_probe_level = 1)
# 검색 파라미터 설정 ---②
search_params=dict(checks=32)
# Flann 매처 생성 ---③
matcher = cv2.FlannBasedMatcher(index_params, search_params)
# 매칭 계산 ---④
matches = matcher.match(descriptor1, descriptor2)
# 매칭 그리기
res = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None, \
            flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)